# Developing a typology of aid donors with quantitative methods: Part II - Clustering

## 1. Introduction

As a reminder, the objective of this two-part project is to present and discuss quantitative methods that can be used to develop a typology of aid donors. We use data from the OECD Development Assistance Committee (DAC). You can find out more about the data in the notebook covering part I.

Part I on Principal Component Analysis (PCA) produced nice-looking 3D scatter plots that give us some clues about which donors should be grouped together and which ones differ from one another. In total, we conducted two PCAs: one on 27 variables with all available donors; another one on the same variables, but only with donors that are member of the DAC. However, we ended the previous part on a cautious note as the results of the PCA were not entirely satisfactory. The share of explained variance in the first 3 principal components used in the 3D plots is not very high. In particular, we used the measure of cosine squared to find that many donors are not well represented in three-dimensional space. This means that there is still great uncertainty around the exact position of donors relative to each other in 3D.

As a preliminary conclusion, we mentioned that the quality of the PCA can still be improved by a more rigorous preparation of the original variables. We will leave this option for a future iteration of this project. For the moment, we focus on how we can use the data coming out of the PCA to get a more reliable result. If three dimensions are not enough to get a good representation, we could add more dimensions from the PCA in order to increase the sum of explained variance. That is why in this second part we'll use more principal components and feed them into a different type of method that might help us with developing a typology: clustering.

Clustering is a form of unsupervised machine learning. Clustering partitions observations into groups (i.e. clusters) with the goal that similar observations end up in the same cluster and dissimilar ones are assigned to other clusters. There are many clustering algorithms. We'll use two of them: 1) agglomarative, hierarchical clustering and 2) Spectral Clustering.

We will proceed according to the following structure:

1. Introduction
2. Data preparation
3. Hierarchical clustering
4. Spectral Clustering
5. Interpretation of the results
6. Conclusion

Note that some parts of the notebook will only be briefly explained; the notebook should be consulted together with the associated blog post.

In [49]:
import pandas as pd
import numpy as np

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.linalg import eigh
from scipy.sparse.csgraph import laplacian

from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score, silhouette_samples

from typing import Iterable, Mapping, Optional, Sequence

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

## 2. Data preparation

Before moving on to clustering, we have to look back at the data that came out of the two PCAs conducted in part I. In the data directory, you can find the two versions of the PCA data (all donors and only-DAC) prepared as csv files:

- `data/pca_data_all.csv`
- `data/pca_data_dac.csv`

The main motivation for taking the transformed data and feeding it into clustering algorithms was that three dimensions were not enough to reliably represent donors in a scatter plot. That is why we'll take more Principal Components (PCs) and conduct clustering on them. Of course, we could also do the clustering on the original variables, but we'll use the PCs.

So, we'll use the PCs. But how many dimensions should we take? To answer that question, let's have look at the following three issues.

- How much explained variance should we retain overall?

- Will the chosen number of principal components be enough to represent each single donor?

- Can the principal components included in the clustering be interpreted in the end (i.e. can we see what they mean and to what original variables they correspond)?

##### For reasons of convenience, this part refers to the data and visualisations already presented in part I. See also the annexe in part I for supporting visualisations of what is discussed in this sub-section.

### 2.1 Explained variance

From the first part we remember that the PCA conducted on all donors only retains roughly 51% of the explained variance in the first three dimensions. The first three dimension of the DAC-only PCA explain 58% of the variance. We could set an arbitrary threshold above which we consider the information from the original variables sufficiently represented (and later reconsider if necessary). Let's set at least 80% of explained variance as the threshold. To know exactly how much explained variance we require, we should also look at the quality of representation of each donor.

### 2.2 Quality of donor representation in the higher dimensional space

In the previous notebook, we measured the quality of representation of individual donors with the sum of their cosine squared. We set a threshold of 0.6 under which we would consider a donor as badly represented with the first 3 PCs. We've seen that the first three PCs are not enough to represent most of the donors well. Well represented were only 18 out of 47 donors in the data set with all donora, and 11 out of 29 in the data set with only DAC members.

From the annexe of part I, we can see that we would need 10 PCs for the data set with all donors and 9 PCs in the DAC-only data set to bring all donors reasonably close to the 0.6 threshold for the cosine squared. With 10 and 9 PCs we would get an explained variance of ca. 87% for both the data set with all donors and the DAC-only data set.

In [50]:
# max number of PCs to be included
max_dims_all = 10 # for data set with all donors
max_dims_dac = 9 # for DAC-only data set

### 2.3 Can the principal components be interpreted?



Finally, we have to face the difficulty that increasing the number of PCs to better represent individual donors makes it more difficult to interpret the PCs. Similarly to how we interpreted the PCs of the 3D scatter plots in part I, we can have a look at the correlations between the original variables and the PCs (see the correlation circles of part I). But with a greater number of PCs we'll find some PCs that are not even moderately correlated with any of the orginal variables.

See the following overview copied from the annexe of part I. 


#### Correlations of original variables with PCs in the data set with all donors (includes both positive and negative correlations).

```
PC1 is at least moderately correlated with  13 orginal variables (with threshold 0.5): 
ODA as percent of GNI (log) --- Europe (%) --- Sub-Saharan Africa (%) --- poor country focus (%) --- Government, Society and Peace (%) --- Agriculture, Forestry, Fishing (%) --- Multi-Sector / Cross-Cutting (%) --- in-donor expenditures (%) --- Budget support and pooled funding (%) --- Multilateral Organisations (%) --- Public Sector (%) --- NGOs, civil society and research institutions (%) --- earmarked-to-core-ratio log
 -------------------- 
PC2 is at least moderately correlated with  7 orginal variables (with threshold 0.5): 
Europe (%) --- Fragile states (%) --- Government, Society and Peace (%) --- Transport, Communication and Energy (%) --- Project-type interventions (%) --- Multilateral Organisations (%) --- share of multilateral
 -------------------- 
PC3 is at least moderately correlated with  4 orginal variables (with threshold 0.5): 
South & Central Asia (%) --- concentration_score --- Humanitarian Aid (%) --- NGOs, civil society and research institutions (%)
 -------------------- 
PC4 is at least moderately correlated with  4 orginal variables (with threshold 0.5): 
Asia-Pacific (%) --- MENA (%) --- Humanitarian Aid (%) --- share of cpa, log
 -------------------- 
PC5 is at least moderately correlated with  1 orginal variables (with threshold 0.5): 
Health, Water, Sanitation (%)
 -------------------- 
PC8 is at least moderately correlated with  1 orginal variables (with threshold 0.5): 
Education (%)
 -------------------- 
```

#### Correlations of original variables with PCs in the data set with DAC donors (includes both positive and negative correlations).

```
PC1 is at least moderately correlated with  15 orginal variables (with threshold 0.5): 
ODA as percent of GNI (log) --- Europe (%) --- Sub-Saharan Africa (%) --- poor country focus (%) --- Fragile states (%) --- Education (%) --- Government, Society and Peace (%) --- Multi-Sector / Cross-Cutting (%) --- Humanitarian Aid (%) --- in-donor expenditures (%) --- Budget support and pooled funding (%) --- Multilateral Organisations (%) --- Public Sector (%) --- NGOs, civil society and research institutions (%) --- earmarked-to-core-ratio log
 -------------------- 
PC2 is at least moderately correlated with  7 orginal variables (with threshold 0.5): 
South & Central Asia (%) --- Asia-Pacific (%) --- Transport, Communication and Energy (%) --- Project-type interventions (%) --- NGOs, civil society and research institutions (%) --- share of multilateral --- share of cpa, log
 -------------------- 
PC3 is at least moderately correlated with  2 orginal variables (with threshold 0.5): 
concentration_score --- MENA (%)
 -------------------- 
PC4 is at least moderately correlated with  1 orginal variables (with threshold 0.5): 
Experts and other technical assistance (%)
 -------------------- 
PC9 is at least moderately correlated with  1 orginal variables (with threshold 0.5): 
Education (%)
 -------------------- 
```

For instance, PC6 and PC7 are not even moderately correlated to any of the original variables in the data set with all donors. In the DAC-only data set PCs 5 to 8 are difficult to interpret.

This could still be improved by going back to the beginning and selecting a different set of original variables and/or deleting some donors that do not really belong to a well defined cluster. But we'll move on for the moment to show a complete iteration of the entire methodological process involved in building a typology. There is no doubt that the result found at the end of this notebook can be improved upon by iterating over these steps again with an improved preprocessing.

#### Approaches for interpreting clusters

Given these problems, we should reflect beforehand how we are going to interpret clusters created from PCs.

When interpreting the results of the clustering, we are going to briefly show and discuss three alternative approaches:

1. Show cluster means on two-dimensional planes for the PCs that we can interpret.
2. Identify the typical representative of a cluster.
3. Compare clusters with regard to original variables using strip plots.

With these preliminary thoughts in mind we can now load the data and go on to the main part about clustering.

In [51]:
# get pca-transformed data from part I: all donors + DAC-only
data_all = pd.read_csv("data/pca_data_all.csv", index_col=0).set_index('Donor')
data_dac = pd.read_csv("data/pca_data_dac.csv", index_col=0).set_index('Donor')

In [52]:
# we also load the original variables for interpration of clusters later
def get_orignal_data() -> pd.DataFrame:
    """
    Load original, unscaled data (i.e. 27 variables) and do some formatting
    to columns that show percentages. 
    """
    original_data = pd.read_csv("data/data_original.csv") # unscaled original data
    for col in original_data.columns:
        if col.startswith('share of') or col.endswith('(%)'):
            original_data[col] = (original_data[col] * 100).round(2)
    return original_data

original_data = get_orignal_data()

## 3. Agglomarative Hierarchical Clustering

We'll use agglomerative, hierarchical clustering as a first step to get an idea of the clusters that might come out of the data. As hierarchical clustering does not require to determine the number of clusters beforehand, it is often used in preparation of other algorithms that require knowledge about the expected number of clusters from the start. In our case, we will use hierarchical clustering before moving on to spectral clustering.  

Agglomerative hierarchical clustering starts with all donors being their own individual cluster. For instance, in the case of the data set with all donors, the algorithm starts with 47 singleton nodes, one for each donor. Then, donors are successively merged into clusters in an iterative process, starting with the two closest nodes. This process is repeated until there is only one large cluster left, which comprises all donors. Variants of this type of clustering vary mainly according to the method used to calculate (dis)similarity between clusters. We'll use Ward's method to iteratively merge nodes. This method minimises the increase in total within-cluster variance at each merging step.

The successive creation of nodes is typically presented in a dendrogram (a hierarchical tree). The tree structure allows to visually explore the clustering process and to decide where to "cut" the tree to know between how many clusters we want to differentiate.

### Dendrogram for all donors

In [53]:
dendro1 = ff.create_dendrogram(
        data_all.iloc[:, :max_dims_all], orientation='left',
        labels=data_all.index,
        linkagefun=lambda x: linkage(
                            data_all.iloc[:, :max_dims_all],
                            'ward', metric='euclidean', optimal_ordering=True
        ))

dendro1.update_layout(
        title=f'Figure 1: Dendrogram with 47 donors (2020)',
        xaxis=dict(title="Distance"),
        width=600,
        height=850,
        font_size=14,
        font_family='Montserrat',
        font_color='black',
        plot_bgcolor='rgb(245, 245, 245)'
    )

dendro1.show()

At the left, all 47 donors are individual nodes. The x-axis starts from zero (because there is no difference within clusters as all clusters are composed of a single donor). Subsequently, the increase on the x-axis indicates the growing difference between members in the same cluster as more and more donors are grouped together. On the right side, all the differences in the data set are contained within a single cluster, and there is no difference anymore between clusters as there is only one cluster left. Note, for instance, that the pairs of donors that are merged first and are therefore closest together are the United Kingdom and the Netherlands, Sweden and Norway, and Canada and Switzerland. Other donors get linked to clusters only very late in the tree (see for example Turkey that is the last donor to be added to a cluster).

The coloring of the dendrogram is determined by the color threshold (the argument `color_threshold` in the `dendrogram` function) where the leaves of the tree are "cut off". Here, we use the default cut-off point as described in the scipy documentation (`0.7*max(Z[:,2])`, see the [docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html)). Usually, the threshold should be placed where intra-cluster differences become larger. The default cut-off leaves us with 3 clusters. We can change the threshold if it makes more sense. The dendrogram allows us to reason about our choice of clusters.

In this particular case, the threshold is not easy to determine as the intra-cluster differences start to shoot up at very different points for the different clusters. For instance, the largest cluster with Sweden, the United Kingdom, etc. seems to be more compact than the cluster with Kuwait, Portugal, Thailand, etc. where the first pairings of donors happen much later. This means the densities of the clusters are very different.

### Dendrogram for DAC-only donors

Here the same for the DAC-only data.

In [54]:
dendro2 = ff.create_dendrogram(
        data_dac.iloc[:, :max_dims_dac], orientation='left',
        labels=data_dac.index,
        linkagefun=lambda x: linkage(
                            data_dac.iloc[:, :max_dims_dac],
                            'ward', metric='euclidean', optimal_ordering=True
        ))

dendro2.update_layout(
        title=f'Figure 4: Dendrogram with DAC donors (2020)',
        xaxis=dict(title="Distance"),
        width=600,
        height=800,
        font_family='Montserrat',
        font_color='black',
        font_size=14,
        plot_bgcolor='rgb(245, 245, 245)'
    )

dendro2.show()

In [55]:
def get_clusters_from_linkage(linkage_data: np.ndarray, donors: Sequence,
                              n_clusters: int) -> dict[str, int]:
    """
    Create dictionary mapping from donors to clusters provided by the dendrogram.

    Requires linkage data, list of donors, and number of clusters.
    """
    mapping = {i: {donor} for i, donor in enumerate(donors)}
    clusters = list(mapping.values())
    for i, row in enumerate(linkage_data, len(donors)):
        mapping[i] = mapping[row[0]].union(mapping[row[1]])
        del mapping[row[0]]
        del mapping[row[1]]
        if len(list(mapping.values())) == n_clusters:
            clusters = list(mapping.values())
    cluster_dict = {donor: i for donor in donors
                             for i, c in enumerate(clusters)
                             if donor in c}
    return cluster_dict

In [56]:
# save results from hierarchical clustering in dict for chosen number of clusters
linkage_data_all = linkage(data_all.iloc[:, :max_dims_all], 'ward', metric='euclidean')
linkage_data_dac = linkage(data_dac.iloc[:, :max_dims_dac], 'ward', metric='euclidean')

hierarchical_clusters_all = get_clusters_from_linkage(linkage_data_all, data_all.index, 3)
hierarchical_clusters_dac = get_clusters_from_linkage(linkage_data_dac, data_dac.index, 3)

In [57]:
# for example, the overview of n clusters from the DAC-only data
print(hierarchical_clusters_dac)

{'Portugal': 1, 'Poland': 0, 'Sweden': 2, 'Italy': 2, 'Austria': 2, 'Greece': 0, 'Slovenia': 0, 'Spain': 2, 'Australia': 1, 'Belgium': 2, 'Norway': 2, 'Ireland': 2, 'Czech Republic': 2, 'Japan': 1, 'Germany': 2, 'Iceland': 2, 'Switzerland': 2, 'United States': 2, 'United Kingdom': 2, 'Denmark': 2, 'Finland': 2, 'France': 2, 'Canada': 2, 'Hungary': 2, 'Slovak Republic': 0, 'Netherlands': 2, 'Luxembourg': 2, 'New Zealand': 1, 'Korea': 1}


### How to choose the number of clusters?

We use hierarchical clustering as a tool to make sensible choices about the number of clusters present in our data. But thinking about the number of clusters depends first and foremost on the question we try to answer. Here, we want to develop a typology of aid donors or at least test if commonly used categorisations of aid donors can be confirmed or refuted. This still leaves open how detailed we want to make the typology. We can be very broad: e.g. 2 or 3 clusters just to find out if DAC and non-DAC donors remain mainly separate or mix with each other. Or we can be very detailed, for instance, finding an intra-DAC donor typology to get a more nuanced picture of this group of donors. 

The two dendrograms show three main clusters with the default color threshold, but we could get a more detailed picture by cutting the trees further to the left; smaller clusters are clearly discernible. For example, 4 or 5 clusters could also be a valid choice in the second dendrogram. We will keep this in mind when using other methods for determining the number of clusters below. 

In view of building a typology, we will also allow the following:
- not all donors need to be part of a cluster. If we hand our data to a clustering algorithm, it will allocate all observations to clusters (unless we use methods like DBSCAN that also identify outliers that don't belong to any clusters). Keeping in mind the goal to build a typology, it is useful to allow that some donors might not fit into a type. Considering some donors as not belonging to a cluster is a valid option.
- small clusters are possible, even just 2 donors might form a cluster (or a donor type).

### Changes to the data sets

The following changes are part of a learning process throughout several iterations of this notebook. The decision to exclude certain donors comes from analysing the dendrograms, but also from using techniques like the silhouette score further below.

Data set with all donors becomes `data_all_reduced`:
- delete some "isolated" donors that could be considered as not being part of any cluster (i.e. donors that are linked to clusters very late in the dendrogram).

DAC-only data set becomes `data_dac_reduced`:
- donors that are very sensitive to varying clustering parameters. In particular, Germany and France could be considered as "swing donors". Due to their central position (see the 3D plots in part I), they get associated with different clusters around them.

In [58]:
# in case you want to exclude certain donors from further analysis
exclude_donors = ["Türkiye", "Kuwait", "Azerbaijan", "Kazakhstan", "Cyprus"]
exclude_donors_dac = ["Germany", "France"]
data_all_reduced = data_all.drop(exclude_donors)
data_dac_reduced = data_dac.drop(exclude_donors_dac)

## 4. Spectral Clustering

Spectral Clustering refers to a family of clustering algorithms that work with data represented as a similarity graph (i.e. as a network of nodes connected by edges representing the similarity between nodes). As stated in [Luxburg (2007), Statistics and Computing, 17](https://arxiv.org/abs/0711.0189): "The problem of clustering can now be reformulated using the similarity graph: we want to find a partition of the graph such that the edges between different groups have very low weights (which means that points in different clusters are dissimilar from each other) and the edges within a group have high weights (which means that points within the same cluster are similar to each other)." In our case, we would create a similarity graph where the nodes represent donors and the edges between donors reflect similarities between them (i.e. between the rows of our data set). This graph can be represented as an adjacency matrix, on which we can apply standard linear algebra operations. Spectral clustering algorithms decompose the adjacency matrix (typically its graph Laplacian) into eigenvectors and eigenvectors. Subsequently, any clustering algorithm (typically KMeans) can be used on the first `k` eigenvectors. See the cited publication above for more details.

My first relflex was to use KMeans as a second clustering algorithm in this project. But KMeans resulted in very instable clusters. This can simply be a sign that there are no clean clusters in the data, which is not unlikely as we do not work with a toy data set designed for clustering. It certainly points to the fact that the shape of clusters is non-convex. This is difficult to handle for KMeans. Spectral clustering has the advantage that it does not make assumptions about the form of clusters.

However, spectral clustering comes with its own challenges. Clusters may vary depending on how we build the similarity matrix and the parameters chosen.

### 4.1 Preparing spectral clustering

We use sklearn's implementation of spectral clustering. To do this, we have to answer three main questions:

#### How to build the similarity graph?

Sklearn offers two standard ways of building the adjacency (or affinity )matrix from our data out of the box (alternatively custom approaches can be precomputed).

A) nearest neighbor graphs: nodes will be connected to their `k` nearest neighbors. The adjacency matrix contains information about connectivity (1: connected; 0: not connected). The result is a sparse matrix.

B) a radial basis function (RBF) kernel (Gaussian kernel) that calculates distances with `np.exp(-gamma * d(X,X) ** 2)` (where `d(X,X)` refers to the euclidean distances of data points). The result is a dense matrix.

#### Which parameters to choose?

A) nearest neighbors require the `n_neighbors` parameter (default=10)
B) RBF requires the `gamma` parameter (default=1.0)


#### How many clusters?

The number of clusters `n_clusters` is a required parameter for spectral clustering (default=8).

There are many methods that can help determine the number of clusters. Most of them are common to all clustering methods. In addition, the **eigengap heuristic** can be used specifically for spectral clustering.

We use the following to determine the number of clusters:

1) Analysis if the results of hierarchical clusterig (see above)

2) Eigengap heuristic: sort eigenvalues in ascending order. Look for the first `k` eigengaps with low values close to zero and the `(k + 1)`th eigenvalue that is considerably larger (the gap is between `k` and `k+1`). Then k is a good guess for the number of clusters.

3) The silhouette score: it measures how close an observation is to observations in its own cluster and how different to observations in the closest different cluster. The score ranges from -1.0 (observation could also be in other cluster) to 1.0 (very good representation in current cluster). We can look at the average and the detailed silhouette score of each donor to assess the quality of clusters.

The next section of the notebook contains the code for all these steps.

In [59]:
def get_silhouette_scores(data: pd.DataFrame, affinity: str,
                          parameters: list[float], cluster_range: list[int])\
                             -> go.Figure:
    """
    Create line chart with silhouette scores for specific affinity method,
    across range of parameters and number of clusters.
    """
    if affinity not in ["nearest_neighbors", "rbf"]:
        raise Exception("You can only use nearest_neighbors or rbf.")
    fig = go.Figure()    
    for p in parameters:
        sil_scores = []

        for c in cluster_range:
            if affinity == "nearest_neighbors":
                spec = SpectralClustering(
                        n_clusters=c, affinity=affinity, n_neighbors=p,
                        n_init=50,
                    ).fit(data)
            else:
                spec = SpectralClustering(
                        n_clusters=c, affinity=affinity, gamma=p, n_init=50,
                    ).fit(data)
            sil = silhouette_score(data, spec.labels_)
            sil_scores.append(sil)

        
        fig.add_trace(go.Scatter(
            x=cluster_range, y=sil_scores, name=p
        ))
    return fig

def get_eig_vals(data: pd.DataFrame, affinity: str, params: list[float])\
                                                 -> dict[float, np.ndarray]:
    """
    Calculate eigenvalues for specific affinity and parameter range of
    spectral clustering.

    Returns dictionary with parameter as key and array of eigenvalues as values.
    """
    if affinity not in ["nearest_neighbors", "rbf"]:
        raise Exception("You can only use nearest_neighbors or rbf.")

    eigval_data = {}
    for p in params:
        if affinity == "nearest_neighbors":
            spec = SpectralClustering(affinity=affinity, n_neighbors=p,
                                      n_init=50).fit(data)
        elif affinity == "rbf":
            spec = SpectralClustering(affinity=affinity, gamma=p,
                                      n_init=50).fit(data)
        laplacian_matrix = laplacian(spec.affinity_matrix_, normed=True)
        try:
            eig_val, eig_vec = eigh(laplacian_matrix.todense())
        except AttributeError:
            eig_val, eig_vec = eigh(laplacian_matrix)
        eigval_data[p] = eig_val
    return eigval_data


def show_eig_gaps(eigen_vals: Mapping[float, np.ndarray],
                  affinity: str, max_vals: int = 15) -> go.Figure:
    """
    Plot eigenvalues for identifying eigengap. Number of sub-plots
    corresponds to number of parameters tested (i.e. number of 
    keys in the mapping provided as first argument).
    """
    para_name = 'n_neighbors' if affinity == "nearest_neighbors" else "gamma"

    fig = make_subplots(
            rows=len(eigen_vals), cols=1,
            subplot_titles=[f"{para_name} {p}" for p in eigen_vals.keys()]
        )

    for i, (p, eig_vals) in enumerate(eigen_vals.items()):

        fig.add_trace(
            go.Scatter(
                    x=list(range(1, len(eig_vals) + 1))[:max_vals],
                    y=eig_vals[:max_vals],
                    mode='markers',
                    hovertemplate="%{x}<extra></extra>"
                ),
            row=i + 1,
            col=1
        )

    fig.update_layout(
        title=f"Eigengap analysis ({affinity})",
        showlegend=False,
        xaxis=dict(title="Eigenvalues"),
        width=500,
        height=len(eigen_vals.keys())*325,
        font_family='Montserrat',
        font_color='black',
        font_size=12,
        plot_bgcolor='rgb(245, 245, 245)'
    )
    return fig


In [60]:
clusters = list(range(2, 11))
n_neighbors = list(range(3, 11))
gammas = np.arange(0.1, 1.1, 0.1) # test values below 1.0 to allow clusters with greater distances between observations

### 4.1.1 Check number of clusters with different parameters for data_all_reduced

#### Eigengap

In [61]:
eigen_vals_all_neighbors = get_eig_vals(data_all_reduced.iloc[:, :max_dims_all],
                                      "nearest_neighbors", n_neighbors)
fig_eig1 = show_eig_gaps(eigen_vals_all_neighbors, "nearest_neighbors")
fig_eig1.show()

In [62]:
eigen_vals_all_rbf = get_eig_vals(data_all_reduced.iloc[:, :max_dims_all],
                                  "rbf", gammas)
fig_eig2 = show_eig_gaps(eigen_vals_all_rbf, "rbf")
fig_eig2.show()

In [63]:
# eigengap figure 1 for blog post

eigen_vals_example1 = get_eig_vals(data_all_reduced.iloc[:, :max_dims_all],
                                   "rbf", [0.4])
eigengap_fig1 = show_eig_gaps(eigen_vals_example1, "rbf")
eigengap_fig1.show()

##### Silhouette Score

In [64]:
fig_sil_all_neighbors = get_silhouette_scores(
                                data_all_reduced.iloc[:, :max_dims_all],
                                affinity="nearest_neighbors",
                                parameters=n_neighbors, cluster_range=clusters
                            )
fig_sil_all_neighbors.show()

In [65]:
fig_sil_all_rbf = get_silhouette_scores(
                            data_all_reduced.iloc[:, :max_dims_all],
                            affinity="rbf", parameters=gammas,
                            cluster_range=clusters
                        )
fig_sil_all_rbf.show()

### 4.1.2 Check number of clusters with different parameters for data_dac_reduced

#### Eigengap

In [66]:
eigen_vals_dac_neighbors = get_eig_vals(
                                data_dac_reduced.iloc[:, :max_dims_dac],
                                "nearest_neighbors", n_neighbors
                            )

fig_eig3 = show_eig_gaps(eigen_vals_dac_neighbors, "nearest_neighbors")
fig_eig3.show()

In [67]:
eigen_vals_dac_rbf = get_eig_vals(
                        data_dac_reduced.iloc[:, :max_dims_dac], "rbf", gammas
                    )
fig_eig4 = show_eig_gaps(eigen_vals_dac_rbf, "rbf")
fig_eig4.show()

In [68]:
# eigengap figure 2 for blog post

eigen_vals_example2 = get_eig_vals(
                            data_dac_reduced.iloc[:, :max_dims_dac],
                            "nearest_neighbors", [3]
                        )

eigengap_fig2 = show_eig_gaps(eigen_vals_example2, "nearest_neighbors")
eigengap_fig2.show() 

In [69]:
fig_sil_dac_neighbors = get_silhouette_scores(
                                data_dac_reduced.iloc[:, :max_dims_dac],
                                affinity="nearest_neighbors",
                                parameters=n_neighbors, cluster_range=clusters
                            )

fig_sil_dac_neighbors.show()

In [70]:
fig_sil_dac_rbf = get_silhouette_scores(
                            data_dac_reduced.iloc[:, :max_dims_dac], affinity="rbf",
                            parameters=gammas, cluster_range=clusters
                    )

fig_sil_dac_rbf.show()

### 4.2. Final clustering

### 4.2.1 Spectral clustering with `data_all_reduced`

In [71]:
n_clusters_all_reduced = 3
affinity_all_reduced = "rbf"
gamma_all = 0.4
n_init = 50

In [72]:
clusterer_all_reduced = SpectralClustering(
                            n_clusters=n_clusters_all_reduced,
                            affinity=affinity_all_reduced, gamma=gamma_all,
                            n_init=n_init
                        ).fit(data_all_reduced.iloc[:, :max_dims_all])

In [73]:
clusters_all_reduced = pd.DataFrame(
                            {
                                'Donor': data_all_reduced.index,
                                'Cluster': clusterer_all_reduced.labels_
                            })

In [74]:
clusters_all_reduced.groupby("Cluster").count()

,Donor
Cluster,
0,14
1,26
2,2


In [75]:
# add column for donor size to DataFrame with clusters
clusters_all_reduced = clusters_all_reduced.merge(
    original_data[['Donor', 'ODA, grant equivalent measure (log)']], on='Donor')

In [76]:
treemap_all_reduced = px.treemap(
                            data_frame=clusters_all_reduced,
                            path=[px.Constant("Clusters"), 'Cluster', 'Donor'],
                            values='ODA, grant equivalent measure (log)',
                            height=1000, width=600,
                            color_discrete_sequence=px.colors.qualitative.Safe,
                            hover_data={
                                'ODA, grant equivalent measure (log)': False
                            }
                        )

treemap_all_reduced.update_layout(
        title="Figure 3: Treemap with clusters of aid donors",
        font_family='Montserrat',
        font_size=14
    )
treemap_all_reduced.show()

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



### 4.2.2 Intra-DAC clusters with `data_dac_reduced`

In [77]:
n_clusters_dac = 5
affinity_dac = "nearest_neighbors"
n_neighbors_dac = 3
n_init=50

In [78]:

clusterer_dac = SpectralClustering(
                    n_clusters=n_clusters_dac,
                    affinity=affinity_dac,
                    n_neighbors=n_neighbors_dac,
                    n_init=n_init
                ).fit(data_dac_reduced.iloc[:, :max_dims_dac])

In [79]:
clusters_dac = pd.DataFrame(
                    {
                        'Donor': data_dac_reduced.index,
                        'Cluster': clusterer_dac.labels_
                    })

In [80]:
clusters_dac.groupby("Cluster").count()

,Donor
Cluster,
0,7
1,5
2,5
3,6
4,4


In [81]:
# add column for donor size to DataFrame with clusters
clusters_dac = clusters_dac.merge(
    original_data[['Donor', 'ODA, grant equivalent measure (log)']], on='Donor')

In [82]:
cluster_color_map = {c: px.colors.qualitative.Safe[c] for c in clusters_dac['Cluster'].unique()}

In [83]:
cluster_color_map

{2: 'rgb(221, 204, 119)',
 4: 'rgb(51, 34, 136)',
 3: 'rgb(17, 119, 51)',
 1: 'rgb(204, 102, 119)',
 0: 'rgb(136, 204, 238)'}

In [102]:
treemap_dac = px.treemap(
                    data_frame=clusters_dac,
                    path=[px.Constant("Clusters"), 'Cluster', 'Donor'],
                    values='ODA, grant equivalent measure (log)',
                    height=1000, width=600,
                    color_discrete_sequence=['rgb(136, 204, 238)', 'rgb(17, 119, 51)', 'rgb(221, 204, 119)', 'rgb(204, 102, 119)', 'rgb(51, 34, 136)'],
                    hover_data={
                          'ODA, grant equivalent measure (log)': False
                    }
                  )

treemap_dac.update_layout(
        title="Figure 6: Treemap with clusters of DAC donors",
        font_family='Montserrat',
        font_size=14
    )

treemap_dac.show()

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



### 4.3 Analyse the silhouette of the clusters

In [85]:
def show_silhouette(data: pd.DataFrame, clusters: np.ndarray,
                    colors: Optional[Mapping[int, str]] = None,
                    title: str = "Silhouette analysis") -> go.Figure:
    
    silhouette_avg = silhouette_score(data, clusters)
    sample_silhouette_values = silhouette_samples(data, clusters)

    overview = pd.DataFrame(
        {
            'Donor': data.index,
            'Cluster': clusters,
            'sil_score': sample_silhouette_values
        })

    fig = go.Figure()

    y_lower = 0
    colors = px.colors.qualitative.Safe if not colors else colors

    for i in sorted(set(clusters)):
        cluster_mask = overview['Cluster'] == i
        cluster_data = overview.loc[cluster_mask, :].sort_values(by='sil_score')

        size_cluster_i = len(cluster_data)
        y_upper = y_lower + size_cluster_i
        
        filled_area = go.Scatter(
            y=np.arange(y_lower, y_upper),
            x=cluster_data['sil_score'],
            mode='lines',
            line=dict(
                    width=0.5,
                    color=colors[i],
                ),
            fill='tozerox',
            name=f"Cluster {str(i)}",
            text=cluster_data['Donor'],
            hovertemplate="%{text}"
        )

        fig.add_trace(filled_area)

        y_lower = y_upper + 2
    
    fig.add_vline(x=silhouette_avg, line_width=1, line_dash="dash",
                  line_color="red")

    fig.update_layout(
        width=500,
        height=600,
        title=title,
        font_family='Montserrat',
        font_color='black',
        font_size=14,
        plot_bgcolor='rgb(245, 245, 245)'
    )

    fig.update_yaxes(showticklabels=False)
    return fig

In [86]:
fig_sil1 = show_silhouette(data_all_reduced.iloc[:, :max_dims_all],
                           clusterer_all_reduced.labels_)
fig_sil1.show()

In [87]:
fig_sil2 = show_silhouette(data_dac_reduced.iloc[:, :max_dims_dac],
                           clusterer_dac.labels_, cluster_color_map,
                           "Figure 7: Silhouette analysis")
fig_sil2.show()

## 5. Interpretation

The section is limited to the interpretation of the reduced data set (data_all_reduced) and the only-DAC clusters.

To interpret the resulting clusters, we use the following:

1) Show cluster means on PCA axes that can be meaningfully interpreted. To be analysed using correlation circles in part I.

2) Show most representative donor of each cluster (the one closest to the cluster mean).

3) Strip plot of clusters with original variables.

In [88]:
original_vars = [
        'ODA grant equivalent as percent of GNI',
        'Europe (%)', 'Asia-Pacific (%)', 'MENA (%)', 'America (%)',
        'Sub-Saharan Africa (%)',
        'poor country focus (%)', 'Fragile states (%)',
        'Land-Locked countries (%)', 'Small island states (%)', 'Education (%)',
        'Health, Water, Sanitation (%)', 'Government, Society and Peace (%)',
        'Transport, Communication and Energy (%)',
        'Agriculture, Forestry, Fishing (%)',
        'Multi-Sector / Cross-Cutting (%)', 'Humanitarian Aid (%)',
        'Experts and other technical assistance (%)',
        'Project-type interventions (%)', 'in-donor expenditures (%)',
        'Budget support and pooled funding (%)',
        'Multilateral Organisations (%)', 'Public Sector (%)',
        'Public-Private Partnerships (PPP) and Private Sector (%)',
        'NGOs, civil society and research institutions (%)',
        'earmarked-to-core-ratio',
        'share of multilateral', 'share of cpa'
    ]


def get_cluster_means(cluster_data: pd.DataFrame, pca_data: pd.DataFrame,
                      max_dims: int) -> pd.DataFrame:
    cluster_cols = [str(i) for i in range(max_dims)] + ['Cluster']
    mean_data = cluster_data.merge(pca_data.reset_index(), on='Donor')
    return mean_data[cluster_cols].groupby('Cluster').mean().reset_index()


def show_cluster_means(cluster_means: pd.DataFrame, show_dims: tuple[int, int],
                       individual_donors: Optional[pd.DataFrame] = None,
                       title: str = "Cluster Means") -> go.Figure:
    """
    Create scatter plot showing cluster means.
    
    Optionally the scatter plot can also show the position of individual donors.
    """
    cluster_means['color'] = cluster_means['Cluster'].apply(lambda x: cluster_color_map[x])
    cluster_means['Cluster'] = cluster_means['Cluster'].astype(str)
    cluster_means['size'] = 20

    x, y = [str(dim) for dim in show_dims]

    fig = go.Figure(data=[
        go.Scatter(
                name="Clusters",
                x=cluster_means[x],
                y=cluster_means[y],
                mode='markers',
                marker=dict(
                        size=cluster_means['size'],
                        color=cluster_means['color'],
                    ),
                text=[f'Cluster {i}' for i in cluster_means['Cluster']],
                hovertemplate='%{text}<extra></extra>',
            ),
    ])

    if individual_donors is not None:
        fig.add_trace(
                go.Scatter(
                    name="Individual donors",
                    x=individual_donors[x],
                    y=individual_donors[y],
                    mode='markers',
                    marker=dict(
                            color=['black']*len(individual_donors.index)
                        ),
                    text=[donor for donor in individual_donors.index],
                    hovertemplate='%{text}<extra></extra>',
                )
        )

    fig.update_layout(
        title=dict(text=f"{title} for PC{int(x) + 1} and PC{int(y) + 1}", font=dict(size=22)),
        width=500,
        height=500,
        font_family='Montserrat',
        font_color='black',
        font_size=10,
        plot_bgcolor='rgb(245, 245, 245)',
    )

    fig.update_xaxes(title=dict(text=f"PC{int(x) + 1}", font=dict(size=16)))
    fig.update_yaxes(title=dict(text=f"PC{int(y) + 1}", font=dict(size=16)))
    return fig

def get_typical_representative(cluster_means: pd.DataFrame,
                               cluster_data: pd.DataFrame,
                               donor_data: pd.DataFrame,
                               max_dims: int) -> dict[int, str]:
    columns = [str(i) for i in range(max_dims)]
    donors = donor_data.merge(cluster_data, on="Donor")

    typical_donors = {}
    for cluster in cluster_means['Cluster'].values:
        cluster_mask = donors['Cluster'] == int(cluster)
        cluster_df = donors.loc[cluster_mask, columns]
        mean_vector = cluster_means[cluster_means['Cluster'] == cluster].loc[:, columns]
        distances = np.linalg.norm(mean_vector.values - cluster_df.values, axis=1)
        min_index = cluster_df.iloc[np.argmin(distances), :].name
        typical_donors[cluster] = donors.loc[min_index, 'Donor']
    return typical_donors


def show_variables(data: pd.DataFrame, vars: Sequence[str]) -> go.Figure:
    """
    Creates an interactive strip plot with original variables of the data set.

    Input data with original variables should contain a 'Cluster' column to
    know to which cluster a donor belongs.  
    """
    n_clusters = len(data['Cluster'].unique())
    visibility = [False]*len(vars)*n_clusters
    visibility[:n_clusters] = [True]*n_clusters

    fig = go.Figure()

    figs = []
    for i, var in enumerate(vars):
        figs.append(
            px.strip(
                    data_frame=data, y=var, x='Cluster', color='Cluster',
                    hover_name='Donor', title=var,
                    color_discrete_map=cluster_color_map,
                    labels={
                        "Cluster": "Cluster",
                        var: var
                    },
                ).update_traces(visible=True if i == 0 else False)
            )

    for chart in figs:
        fig.add_traces(list(chart.select_traces()))

    fig.update_layout(
        showlegend=False,
        width=600,
        updatemenus=[
            {
                "buttons": [
                    {
                        "label": col,
                        "method": "update",
                        "args": [
                            {"visible": [(i >= vars.index(col)*n_clusters) \
                                        & (i < vars.index(col)*n_clusters + n_clusters)\
                                        for i, _ in enumerate(visibility)]},
                            {"title": f"{col}"},
                        ],
                    }
                    for col in vars
                ],
                'active': 0,
                'x': 0.5,
                'xanchor': "center",
                'y': 1.15,
                'yanchor': "top"
            }
        ]
    )

    fig.update_xaxes(title="Cluster Nr.")
    fig.update_yaxes(title="value")
    return fig

In [89]:
# variables for strip plot to be published in post
# reduced number of variables
original_vars_reduced = [
        'ODA grant equivalent as percent of GNI',
        'Europe (%)', 'Asia-Pacific (%)', 'MENA (%)',
        'Sub-Saharan Africa (%)',
        'poor country focus (%)', 'Fragile states (%)',
        'Land-Locked countries (%)', 'Small island states (%)', 'Education (%)',
        'Health, Water, Sanitation (%)', 'Government, Society and Peace (%)',
        'Transport, Communication and Energy (%)',
        'Agriculture, Forestry, Fishing (%)',
        'Multi-Sector / Cross-Cutting (%)', 'Humanitarian Aid (%)',
        'Experts and other technical assistance (%)',
        'Project-type interventions (%)', 'in-donor expenditures (%)',
        'Budget support and pooled funding (%)', 'Public Sector (%)',
        'Public-Private Partnerships (PPP) and Private Sector (%)',
        'NGOs, civil society and research institutions (%)',
        'earmarked-to-core-ratio',
        'share of multilateral', 'share of cpa'
    ]

### 5.1 Interpretaion of clusters for data_all_reduced

#### 5.1.1 Cluster means for PC1 and PC3

In [90]:
cluster_means_all = get_cluster_means(clusters_all_reduced,
                                      data_all_reduced.iloc[:, :max_dims_all],
                                      max_dims_all)

fig = show_cluster_means(cluster_means_all, (0, 2))
fig.show()

#### 5.1.2 Most typical representatives for data_all_reduced

In [91]:
typical_donors = get_typical_representative(
                        cluster_means_all, clusters_all_reduced,
                        data_all_reduced.iloc[:, :max_dims_all], max_dims_all
                    )

for cluster, donor in typical_donors.items():
    print(f"The donor closest to the mean in cluster {cluster} is {donor}.")

The donor closest to the mean in cluster 0 is Slovak Republic.
The donor closest to the mean in cluster 1 is Switzerland.
The donor closest to the mean in cluster 2 is Australia.


#### 5.1.3 Analysis of original variables

In [92]:
original_vars_all = original_data.merge(clusters_all_reduced, on='Donor', how='inner')
var_fig_all = show_variables(original_vars_all, original_vars)

var_fig_all.show()

### 5.2 Interpretation for DAC-only dataset

#### 5.2.1 Cluster means

In [93]:
cluster_means_dac = get_cluster_means(clusters_dac,
                                      data_dac_reduced.iloc[:, :max_dims_dac],
                                      max_dims_dac)

cluster_fig_dac = show_cluster_means(cluster_means_dac, (0, 1),
                         data_all.loc[["France", "Germany"], :],
                         title = "Figure 8: Cluster Means")
cluster_fig_dac.show()

#### 5.2.2 Most typical representative

In [94]:
typical_donors_dac = get_typical_representative(
                        cluster_means_dac, clusters_dac,
                        data_dac_reduced.iloc[:, :max_dims_dac], max_dims_dac
                    )

for cluster, donor in typical_donors_dac.items():
    print(f"The donor closest to the mean in cluster {cluster} is {donor}.")

The donor closest to the mean in cluster 0 is Switzerland.
The donor closest to the mean in cluster 1 is Italy.
The donor closest to the mean in cluster 2 is Korea.
The donor closest to the mean in cluster 3 is Sweden.
The donor closest to the mean in cluster 4 is Slovenia.


#### 5.2.3 Analysis of original variables

In [95]:
original_vars_dac = original_data.merge(clusters_dac, on='Donor', how='inner')
var_fig_dac = show_variables(original_vars_dac, original_vars)

var_fig_dac.show()

In [96]:
# strip plot for publication in blog post with reduced number of variables
original_vars_dac2 = original_data.merge(clusters_dac, on='Donor', how='inner')
var_fig_dac2 = show_variables(original_vars_dac, original_vars_reduced)

var_fig_dac2.show()

## 6. Conclusion and ideas for improvement

This concludes the two-part project about using quantitative methods for developing a typology of aid donors. The final clustering might not yet be a robust typology. After all, we would have to do more to relate this project to a specific research question. However, the two parts of this post illustrate a process that can be applied to develop such a typology, or at least confirm or refute commonly used categorisations of aid donors. More iterations of this process would be necessary to arrive at a more advanced solution. It is also important to recognize that there is no guarantee to find a perfectly clean clustering structure as we are working with a real world data set.

Among the various steps in the process, I would highlight three aspects that deserve particular attention as they influence the clustering result the most:

First, the selection and transformation of variables can have a strong impact on the final clustering result. Variable selection presupposes a rigorous literature review and the formulation of a specific research question. This is the most time-consuming part of the work. Part I already contains a long section about data exploration; a more exhaustive implementation is beyond the scope of a blog post. Investing more work in the preparation of the data can improve the results of the PCA by storing more of the explained variance in fewer PCs and making the clustering result more interpretable.

Second, the analysis of the complete data set with all available donors struggled with the heterogeneity of donors. The initial data set is composed of DAC members, "Participants" and other donors reporting data to the DAC. It was difficult to integrate the largest number of donors possible without getting a result that simply opposes "core" DAC donors versus the rest. That is why the intra-DAC clustering yielded more interesting results. This boils down to the challenge of identifying outliers without simply considering all non-DAC donors as outliers.

Third, the choice of the clustering method and the tuning of relevant parameters obviously influences the clustering result. As an additional step, sensitivity analysis with regard to alternative clustering solutions could improve the quality of the result. In this post, we have touched upon this by omitting France and Germany from the intra-DAC clustering to see if we get a more stable result. This could be done more systematically and would help to see what groups of donors persist across changes applied through different clustering methods and parameters.